In [9]:
import os

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', 100)

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
from matplotlib import pylab as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

/Users/karfly/tf_env/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
from dota_ml.data import data_url, transform_data
from dota_ml.utils import generate_grid, make_submission, plot_feature_ranking, plot_feature_ranking

In [3]:
if not os.path.exists('data/'):
    !wget $data_url -q --show-progress
    !tar -xvf data.tar.gz
else:
    print('Data already exists!')

Data already exists!


In [7]:
data_params = {
    'gold_features': True,
    'lh_features': True,
    'xp_features': True,
#     'add_heroes_by_player': True, 
    'add_heroes_by_team': True
}

train_df, test_df = transform_data(**data_params)

Adding one-hot encoding hero features...


In [ ]:
X_train = train_df.drop('radiant_won', axis=1)
y_train = train_df['radiant_won']

In [ ]:
model_param_grid = {
    'n_estimators': [5000],
    'max_depth': [None, 6, 10],
#     'max_features': [None, 'sqrt', 'log2'],
    'min_samples_leaf': [1, 2, 5, 100],
    'criterion': ['gini', 'entropy'],

    'random_state': [0],
}

In [ ]:
n_iter = 5
scoring = 'roc_auc'
k_folds = 3

gs = RandomizedSearchCV(RandomForestClassifier(), model_param_grid,
                        scoring=scoring, cv=k_folds, n_iter=n_iter,
                        refit=True, n_jobs=-1, verbose=1)
gs.fit(X_train, y_train)

In [ ]:
for score, std, params in zip(gs.cv_results_['mean_test_score'],
                              gs.cv_results_['std_test_score'],
                              gs.cv_results_['params']):
    print('- score={:.5}, std={:.5} | params={}'.format(score, std, params))

In [ ]:
best_score = gs.best_score_
best_estimator = gs.best_estimator_
best_params = gs.best_params_

print('best_score: {}'.format(best_score))
print('best params: {}'.format(best_params))

In [ ]:
make_submission(test_df, best_estimator,
                'submissions/', 'rfc', {**data_params, **best_params}, best_score)

In [ ]:
plot_feature_ranking(best_estimator.feature_importances_, test_df.columns, max_n_importances=50)

---